
Input layer - 784 nodes
1st layer - 10 nodes (representing each digit)
2nd layer - 10 nodes

3 parts to training network
Forward propagation - take image and run throug the network
Back propagation - backwards pass to calculate how much error we get 
Update variables - figure out how much we need to nudge the weights and biases



In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [6]:
data = pd.read_csv('dataset/train.csv')

In [7]:
data.head()

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  
3         0         0         0         0  
4         0         0         0         0  

[5 rows x 785 columns]

In [87]:
data = np.array(data)
m, n = data.shape # m is the rows, n is the featues + 1 (because of the label)
np.random.shuffle(data)


data_dev = data[0:1000].T 
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [88]:
X_train.shape

(784, 41000)

In [89]:
# Initialise parameters

def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) # (number of samples, number of classes)
    one_hot_Y[np.arange(Y.size), Y] = 1 
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [93]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2,W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 200 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [94]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 2000, 0.1)


Iteration:  0
[5 2 2 ... 6 1 6] [7 6 4 ... 9 5 8]
Accuracy:  0.17082926829268294
Iteration:  200
[7 6 4 ... 4 5 3] [7 6 4 ... 9 5 8]
Accuracy:  0.7352195121951219
Iteration:  400
[7 6 4 ... 4 5 3] [7 6 4 ... 9 5 8]
Accuracy:  0.8196341463414634
Iteration:  600
[7 6 4 ... 4 5 3] [7 6 4 ... 9 5 8]
Accuracy:  0.8535121951219512
Iteration:  800
[7 6 4 ... 9 5 9] [7 6 4 ... 9 5 8]
Accuracy:  0.8694390243902439
Iteration:  1000
[7 6 4 ... 9 5 9] [7 6 4 ... 9 5 8]
Accuracy:  0.8781463414634146
Iteration:  1200
[7 6 4 ... 9 5 9] [7 6 4 ... 9 5 8]
Accuracy:  0.8845853658536585
Iteration:  1400
[7 6 4 ... 9 5 9] [7 6 4 ... 9 5 8]
Accuracy:  0.8893170731707317
Iteration:  1600
[7 6 4 ... 9 5 9] [7 6 4 ... 9 5 8]
Accuracy:  0.893
Iteration:  1800
[7 6 4 ... 9 5 9] [7 6 4 ... 9 5 8]
Accuracy:  0.8961707317073171


Credit to Samson Zhang